# Protótipo de Deep Learning para o descobrimento de novos peptídeos
- Olá, o objetivo desse trabalho foi desenvolver uma rede neural usando python, pytorch e scikit-learn a fim de avaliar a possibilidade de descoberta de novos peptídeos antimicrobianos.
- O código foi estruturado em etapas, desde o tratamento dos dados até a avaliação de novas sequências. As variáveis foram mantidas em inglês para se aproximar com a linguagem das documentações.

# Tratamento inicial dos dados

- Sequências APD, DBAASP e DRAMP como sequências positivas (AMP).
- Sequências do Uniprot como não-AMP.
  - Ao procurar por protein no Uniprot, obtemos uma lista muito extensa de sequências. Sendo assim, foi aplicado um filtro 'protein NOT antimicrobial NOT antibiotic NOT antibacterial NOT antiviral NOT antifungal NOT antimalarial NOT antiparasitic NOT anti-protist NOT anticancer NOT defensin NOT defense NOT cathelicidin NOT histatin NOT bacteriocin NOT microbicidal NOT fungicide NOT signal NOT recombination NOT ribosomal NOT secretion NOT DNA NOT RNA AND (length:[* TO 190])'. Com esse filtro, foram obtidas 11.165 sequências (referência https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-022-08310-4#Sec9).

In [ ]:
import re
import pandas as pd
import warnings
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from collections import OrderedDict
from google.colab import files
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
%matplotlib inline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")     # Verificação para saber se há uma GPU disponível

In [ ]:
def calc_hidrofob(sequence):
  indx_hidro = {
      'A': 0.310,
      'R': -1.010,
      'N': -0.600,
      'D': -0.770,
      'C': 1.540,
      'Q': -0.220,
      'E': -0.640,
      'G': 0.000,
      'H': 0.130,
      'I': 1.800,
      'L': 1.700,
      'K': -0.990,
      'M': 1.230,
      'F': 1.790,
      'P': 0.720,
      'S': -0.040,
      'T': 0.260,
      'W': 2.250,
      'Y': 0.960,
      'V': 1.220
      }

  if isinstance(sequence, str):
    sum_hydrophobicity = sum(indx_hidro.get(aa, 0) for aa in sequence)
    return sum_hydrophobicity / len(sequence)
  else:
    return 0


def calc_net_charge(sequence):
  if isinstance(sequence, str):
    count_positives = sum(1 for aa in sequence if aa in ['K', 'R'])
    count_negatives = sum(1 for aa in sequence if aa in ['D', 'E'])
    net_charge = count_positives - count_negatives
    return net_charge
  else:
    return 0

warnings.filterwarnings("ignore")
unificated_sequences = []
non_amp_sequences = []
sequencias_filtradas = []
fully_sequences_dataset = []

dbaasp = "peptides_dbaasp_2-190_all_modificated_targets.csv"
apd = "APD_sequence_release_09142020.txt"
uniprot_swiss = "uniprotkb_protein_NOT_antimicrobial_NOT_2024_05_14.xlsx"
comprimento_minimo = 2
comprimento_maximo = 190
dados_excel_uniprot = pd.read_excel(uniprot_swiss, engine='openpyxl')

for index, linha in dados_excel_uniprot.iterrows():
    if comprimento_minimo <= linha['Length'] <= comprimento_maximo:
        sequencias_filtradas.append(linha['Sequence'])

# print(f'Sequências do UNIPROT filtradas: {len(sequencias_filtradas)}')
peptides_dbaasp = pd.read_csv(dbaasp)
sequences_dbaasp = peptides_dbaasp["SEQUENCE"]

for sequence_dbaasp in sequences_dbaasp:
    unificated_sequences.append(sequence_dbaasp)

try:
    with open(f"{apd}", "r") as arquivo:
        data = arquivo.read()
        apd_regex = r'(?<=\))\s*([A-Z\s]+)(?=\s*[A-Z]*$)'
        sequences_apd = re.findall(apd_regex, data, re.M)

        for sequence in sequences_apd:
            # print(f'{sequence} \t {len(sequence)}')
            if unificated_sequences[0] is None:
                print("Epa, veio nada")
            else:
                unificated_sequences.append(sequence)
except FileNotFoundError:
    print(f"O arquivo {apd} não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")
# todas as sequências agora estão em unificated_sequences

unificated_sequences_no_duplicate = list(OrderedDict.fromkeys(unificated_sequences))
print(f'Número de sequências não duplicadas: {len(unificated_sequences_no_duplicate)}')

seq_filt_sem_correspondencias = [seq for seq in sequencias_filtradas if seq not in unificated_sequences_no_duplicate]
print(f'Sequências sem correspondência com AMP: {len(seq_filt_sem_correspondencias)}')

for sequence_non_amp in seq_filt_sem_correspondencias:
    non_amp_sequences.append(sequence_non_amp)

print(f'Número de sequências não-AMP: {len(non_amp_sequences)}')

amp_hydrophobicity = [calc_hidrofob(seq) for seq in unificated_sequences_no_duplicate]
amp_charge = [calc_net_charge(seq) for seq in unificated_sequences_no_duplicate]
non_amp_hydrophobicity = [calc_hidrofob(seq) for seq in non_amp_sequences]
non_amp_charge = [calc_net_charge(seq) for seq in non_amp_sequences]

# Criando um dataframe contendo as sequências, hidrofobicidade, carga e classificação
amp_data = pd.DataFrame({
    'Sequence': unificated_sequences_no_duplicate,
    'Hydrophobicity': amp_hydrophobicity,
    'Charge': amp_charge,
    'Classification': '1.0'
})

non_amp_data = pd.DataFrame({
    'Sequence': non_amp_sequences,
    'Hydrophobicity': non_amp_hydrophobicity,
    'Charge': non_amp_charge,
    'Classification': '0.0'
})

# Número de sequências não duplicadas: 15220
# Sequências sem correspondência com AMP: 11127
# Número de sequências não-AMP: 11127  - 3709 sequências se quisermos dividir em 3 dataframes

fully_sequences_dataset = pd.concat([amp_data, non_amp_data], ignore_index=True)
shuffled_fully_sequences_dataset = fully_sequences_dataset.sample(frac=1).reset_index(drop=True)
shuffled_fully_sequences_dataset


# Verificação de maior comprimento de uma sequência AMP - encontrado 190 na execução atual

# maior_comprimento = 0
# with open("Verify.txt", "w") as ver:
#     ver.write(f"Sequences \t Lenght\n")

#     for c in unificated_sequences_no_duplicate:
#         if isinstance(c, str):
#             ver.write(f"{c} \t {len(c)}\n")
#             comprimento_atual = len(c)
#             if comprimento_atual > maior_comprimento:
#                 maior_comprimento = comprimento_atual

# print(f'Maior_comprimento_AMP: {maior_comprimento}')

Sequências do UNIPROT filtradas: 11165
Número de sequências não duplicadas: 15220
Sequências sem correspondência com AMP: 11127
Número de sequências não-AMP: 11127


,Sequence,Hydrophobicity,Charge,Classification
0,MNFKYIKKDG,0.144000,2,0.0
1,MASESKDSPSNNPGLHATPDEATKGYFLQQTMFRIKDPKVSLEFYS...,0.325730,-6,0.0
2,GFIFH,1.102000,0,1.0
3,GACAARCRLSSRPRLCHR,0.259444,5,1.0
4,VDKPPYLPRYRPPRRIYNR,0.199474,5,1.0
...,...,...,...,...
26342,ALWKNMLKGIGKLAGQAALGAVKTLVGA,0.484286,4,1.0
26343,TIKVPEGFDYELYNRNDINRYVDAVLTIPKVICDHLGLGVKTGLPY...,0.464333,1,0.0
26344,GEFLKCGESCVQGECYTPGCSCDWPICKKN,0.470000,-1,1.0
26345,RVCIRVCRNGVCYRRCW,0.540000,5,1.0


In [ ]:
for index, item in enumerate(shuffled_fully_sequences_dataset['Sequence']):
    tipo = type(item)
    if tipo is not str:
        print(f"Índice: {index}, Conteúdo: {item}, Tipo: {tipo}")

In [ ]:
shuffled_fully_sequences_dataset.dropna(inplace=True)
shuffled_fully_sequences_dataset

,Sequence,Hydrophobicity,Charge,Classification
0,MNFKYIKKDG,0.144000,2,0.0
1,MASESKDSPSNNPGLHATPDEATKGYFLQQTMFRIKDPKVSLEFYS...,0.325730,-6,0.0
2,GFIFH,1.102000,0,1.0
3,GACAARCRLSSRPRLCHR,0.259444,5,1.0
4,VDKPPYLPRYRPPRRIYNR,0.199474,5,1.0
...,...,...,...,...
26342,ALWKNMLKGIGKLAGQAALGAVKTLVGA,0.484286,4,1.0
26343,TIKVPEGFDYELYNRNDINRYVDAVLTIPKVICDHLGLGVKTGLPY...,0.464333,1,0.0
26344,GEFLKCGESCVQGECYTPGCSCDWPICKKN,0.470000,-1,1.0
26345,RVCIRVCRNGVCYRRCW,0.540000,5,1.0


In [ ]:
def uppercase_sequence(sequence):
    return sequence.upper()

# Aplicar a função em todas as sequências presentes no df
shuffled_fully_sequences_dataset['Sequence'] = shuffled_fully_sequences_dataset['Sequence'].apply(uppercase_sequence)

In [ ]:
# Função para transformar as sequências em codificação one_hot
def clean_sequence(sequence):
    valid_characters = set('ACDEFGHIKLMNPQRSTVWY')
    cleaned_sequence = ''.join([char for char in sequence if char in valid_characters])
    return cleaned_sequence

def one_hot_encoding(peptide_sequence):
    amino_acid_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4,
                        'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9,
                        'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14,
                        'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}

    cleaned_sequence = clean_sequence(peptide_sequence)
    comp_peptide = len(cleaned_sequence)
    one_hot_encoded = np.zeros((comp_peptide, 20))

    # Preenchendo a matriz com 1 na posição referente ao aminoácido
    for i, amino_acid in enumerate(cleaned_sequence):
        if amino_acid in amino_acid_index:
            index = amino_acid_index[amino_acid]
            one_hot_encoded[i, index] = 1

    return one_hot_encoded

In [ ]:
shuffled_fully_sequences_dataset['Sequence'] = shuffled_fully_sequences_dataset['Sequence'].astype(str)
shuffled_fully_sequences_dataset['Sequence'] = shuffled_fully_sequences_dataset['Sequence'].apply(one_hot_encoding)
shuffled_fully_sequences_dataset['Sequence']

0        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
2        [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...
3        [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...
4        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
                               ...                        
26342    [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
26343    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
26344    [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...
26345    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
26346    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...
Name: Sequence, Length: 26347, dtype: object

In [ ]:
def transform_tensor(fraction):
    # Se o valor for único, converte para um tensor
    if isinstance(fraction, float) or isinstance(fraction, int) or isinstance(fraction, str):
        return torch.tensor(fraction, dtype=torch.float32)
    # Se for uma coleção de valores, converte cada item para um tensor e retorna a lista de tensores
    elif isinstance(fraction, list):
        return [torch.tensor(item, dtype=torch.float32) for item in fraction]
    # Se for uma matriz, converte para tensor
    elif isinstance(fraction, np.ndarray):
        return torch.tensor(fraction.tolist(), dtype=torch.float32)
    else:
        raise TypeError("Tipo de valor não suportado: ", type(fraction))

In [ ]:
shuffled_fully_sequences_dataset['Sequence'] = shuffled_fully_sequences_dataset['Sequence'].apply(transform_tensor)
shuffled_fully_sequences_dataset['Sequence']

0        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
1        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
2        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
3        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
4        [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
                               ...                        
26342    [[tensor(1.), tensor(0.), tensor(0.), tensor(0...
26343    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
26344    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
26345    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
26346    [[tensor(0.), tensor(0.), tensor(0.), tensor(0...
Name: Sequence, Length: 26347, dtype: object

In [ ]:
shuffled_fully_sequences_dataset['Sequence'][0][0]


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [ ]:
shuffled_fully_sequences_dataset['Hydrophobicity'] = shuffled_fully_sequences_dataset['Hydrophobicity'].apply(transform_tensor)
shuffled_fully_sequences_dataset['Hydrophobicity']

0        tensor(0.1440)
1        tensor(0.3257)
2        tensor(1.1020)
3        tensor(0.2594)
4        tensor(0.1995)
              ...      
26342    tensor(0.4843)
26343    tensor(0.4643)
26344    tensor(0.4700)
26345    tensor(0.5400)
26346    tensor(0.1878)
Name: Hydrophobicity, Length: 26347, dtype: object

In [ ]:
shuffled_fully_sequences_dataset['Charge'] = shuffled_fully_sequences_dataset['Charge'].apply(transform_tensor)
shuffled_fully_sequences_dataset['Charge']

0         tensor(2.)
1        tensor(-6.)
2         tensor(0.)
3         tensor(5.)
4         tensor(5.)
            ...     
26342     tensor(4.)
26343     tensor(1.)
26344    tensor(-1.)
26345     tensor(5.)
26346    tensor(12.)
Name: Charge, Length: 26347, dtype: object

In [ ]:
shuffled_fully_sequences_dataset['Classification'] = shuffled_fully_sequences_dataset['Classification'].astype(float)
shuffled_fully_sequences_dataset['Classification'] = shuffled_fully_sequences_dataset['Classification'].apply(transform_tensor)
shuffled_fully_sequences_dataset['Classification']

0        tensor(0.)
1        tensor(0.)
2        tensor(1.)
3        tensor(1.)
4        tensor(1.)
            ...    
26342    tensor(1.)
26343    tensor(0.)
26344    tensor(1.)
26345    tensor(1.)
26346    tensor(1.)
Name: Classification, Length: 26347, dtype: object

In [ ]:
print(shuffled_fully_sequences_dataset)

                                                Sequence  Hydrophobicity  \
0      [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.1440)   
1      [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.3257)   
2      [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(1.1020)   
3      [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.2594)   
4      [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.1995)   
...                                                  ...             ...   
26342  [[tensor(1.), tensor(0.), tensor(0.), tensor(0...  tensor(0.4843)   
26343  [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.4643)   
26344  [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.4700)   
26345  [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.5400)   
26346  [[tensor(0.), tensor(0.), tensor(0.), tensor(0...  tensor(0.1878)   

            Charge Classification  
0       tensor(2.)     tensor(0.)  
1      tensor(-

# Criação do modelo de rede neural com Pytorch

Link de referência para a criação de uma rede LSTM (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

- Para a criação, utilizaremos o Pytorch. Poderia utilizar o TensorFlow e Keras? Poderia, mas nesse caso vamos usar uma tecnologia mais recente e que tem se mostrado bem eficiente (o GPT tem uma base no Pytorch).
- Como vimos, temos 15628 sequências em matrizes. Alguns autores optariam por fazer uma rede neural com convolução, que nada mais é do que a união das matrizes dos dados, porém, ao longo do processo perdemos parte desses dados, que, no nosso caso, não é interessante, uma vez que se reduzirmos a codificação one_hot, perderemos os dados dos aminoácidos iniciais.

In [ ]:
class PepModelLSTM(nn.Module):
    def __init__(self, input_size=20, hidden_size=10, num_layers=1, num_classes=2):
        super(PepModelLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size).to(x.device)   # Enviando os dados para o device, seja cuda ou CPU
        c0 = torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))  # out: Tupla do tensor representada por (batch_size, seq_length, 2 * hidden_size) devido à bidirecionalidade

        # Considerando apenas a saída do último passo de tempo e descartando a outra direção
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Exibir as primeiras amostras de cada coluna para confirmar se os dados estão certos

# for col in shuffled_fully_sequences_dataset.columns:
#     print(f"Primeiras amostras da coluna '{col}':")
#     for sample in shuffled_fully_sequences_dataset[col][:5]:
#         print(sample)

In [ ]:
X = shuffled_fully_sequences_dataset['Sequence']
y = shuffled_fully_sequences_dataset['Classification']
print(X.shape)
print(y.shape)

(26347,)
(26347,)


In [ ]:
def pad_sequences(sequences, max_length):
    padded_sequences = []
    empty_indices = []
    for i, seq in enumerate(sequences):
        seq_len = len(seq)
        if seq_len == 0:
            empty_indices.append(i)
            padded_sequences.append(seq)
        elif seq_len <= max_length:
            padding = torch.zeros(max_length - seq_len, seq.shape[1])
            padded_seq = torch.cat([seq, padding])
            padded_sequences.append(padded_seq)
        else:
            padded_sequences.append(seq[:max_length])
    return padded_sequences, empty_indices

In [ ]:
max_sequence_length = 190  # Comprimento máximo encontrado
X_padded, empty_indices = pad_sequences(X.values, max_sequence_length)
print("Índices das sequências vazias:", empty_indices)

Índices das sequências vazias: []


In [ ]:
y = y.astype(np.float32)
X_tensor = torch.stack(X_padded).to(device)
y_tensor = torch.tensor(y.values, dtype=torch.float32).to(device)

In [ ]:
saved_model = 'pep_lstm_re_trained.pth'
model = PepModelLSTM().to(device)
model.load_state_dict(torch.load(saved_model))

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss().to(device)
epochs = 20000
losses = []

num_classes = 2
y_tensor_one_hot = F.one_hot(y_tensor.to(torch.int64), num_classes)

In [ ]:
for i in range(epochs):
  model.train()
  optimizer.zero_grad()
  y_pred = model.forward(X_tensor)
  loss = criterion(y_pred, y_tensor_one_hot.float())

  loss.backward()
  optimizer.step()
  losses.append(loss)
  if i % 100 == 0:
    print(f'Epoch: {i} and loss: {loss:.6f}')

Epoch: 0 and loss: 0.991757
Epoch: 100 and loss: 0.443921
Epoch: 200 and loss: 0.443676
Epoch: 300 and loss: 0.448649
Epoch: 400 and loss: 0.578443
Epoch: 500 and loss: 0.506161
Epoch: 600 and loss: 0.494112
Epoch: 700 and loss: 0.492126
Epoch: 800 and loss: 0.491307
Epoch: 900 and loss: 0.449266
Epoch: 1000 and loss: 0.447495
Epoch: 1100 and loss: 0.445366
Epoch: 1200 and loss: 0.444484
Epoch: 1300 and loss: 0.475052
Epoch: 1400 and loss: 0.456584
Epoch: 1500 and loss: 0.453259
Epoch: 1600 and loss: 0.461414
Epoch: 1700 and loss: 0.445277
Epoch: 1800 and loss: 0.445096
Epoch: 1900 and loss: 0.439038
Epoch: 2000 and loss: 0.441031
Epoch: 2100 and loss: 0.432822
Epoch: 2200 and loss: 0.450489
Epoch: 2300 and loss: 0.433354
Epoch: 2400 and loss: 0.433586
Epoch: 2500 and loss: 0.429801
Epoch: 2600 and loss: 0.431461
Epoch: 2700 and loss: 0.432158
Epoch: 2800 and loss: 0.427380
Epoch: 2900 and loss: 0.424495
Epoch: 3000 and loss: 0.421205
Epoch: 3100 and loss: 0.419246
Epoch: 3200 and loss

In [ ]:
correct = 0
total = 0

model.eval()    # Modo de avaliação

with torch.no_grad():
    for i, data in enumerate(X_tensor):
        y_val = model(data.unsqueeze(0))
        predicted_class = y_val.argmax().item()
        true_class = y_tensor[i].item()

        print(f'{i+1}.) Predicted: {predicted_class}, True: {true_class}')

        if predicted_class == true_class:
            correct += 1
        total += 1

print(f'Nós temos {correct} previsões corretas em um total de {total} sequências.')

A saída de streaming foi truncada nas últimas 5000 linhas.
21349.) Predicted: 1, True: 1.0
21350.) Predicted: 1, True: 1.0
21351.) Predicted: 1, True: 1.0
21352.) Predicted: 0, True: 0.0
21353.) Predicted: 0, True: 0.0
21354.) Predicted: 1, True: 1.0
21355.) Predicted: 1, True: 1.0
21356.) Predicted: 1, True: 1.0
21357.) Predicted: 0, True: 0.0
21358.) Predicted: 1, True: 1.0
21359.) Predicted: 0, True: 0.0
21360.) Predicted: 1, True: 1.0
21361.) Predicted: 1, True: 1.0
21362.) Predicted: 0, True: 0.0
21363.) Predicted: 0, True: 0.0
21364.) Predicted: 1, True: 1.0
21365.) Predicted: 1, True: 1.0
21366.) Predicted: 1, True: 1.0
21367.) Predicted: 1, True: 1.0
21368.) Predicted: 1, True: 1.0
21369.) Predicted: 1, True: 1.0
21370.) Predicted: 1, True: 1.0
21371.) Predicted: 0, True: 0.0
21372.) Predicted: 0, True: 0.0
21373.) Predicted: 0, True: 0.0
21374.) Predicted: 0, True: 0.0
21375.) Predicted: 1, True: 1.0
21376.) Predicted: 0, True: 1.0
21377.) Predicted: 1, True: 1.0
21378.) Predi

In [ ]:
model.eval()    # Modo de avaliação

# Lista para armazenar os dados previstos e reais
predicted_labels = []
true_labels = []
predicted_probabilities = []

with torch.no_grad():
    for i, data in enumerate(X_tensor):
        y_val = model(data.unsqueeze(0))

        # Saída da rede neural
        predicted_class = y_val.argmax().item()
        # Probabilidade de ser 1.0
        predicted_probability = torch.sigmoid(y_val[:, 1]).item()
        predicted_probabilities.append(predicted_probability)
        # Saída real
        true_class = y_tensor[i].item()
        # Armazenando as previsões e os verdadeiros
        predicted_labels.append(predicted_class)
        true_labels.append(true_class)

# Calculando as métricas do sk-learn
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)
auroc = roc_auc_score(true_labels, predicted_probabilities)
accuracy = accuracy_score(true_labels, predicted_labels)
gini_index = (2 * auroc) - 1

# Exibindo os resultados
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'AUROC: {auroc:.4f}')
print(f'Acurácia: {accuracy:.4f}')
print(f'Índice de Gini: {gini_index:.4f}')

Precisão: 0.8048
Recall: 0.9525
F1-Score: 0.8724
AUROC: 0.8451
Acurácia: 0.8391
Índice de Gini: 0.6901


In [ ]:
losses = [loss.cpu().item() for loss in losses]

plt.plot(range(epochs), losses)
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
std_deviation = np.std(losses)
print(f"Desvio padrão das perdas: {std_deviation:.3f}")

Desvio padrão das perdas: 0.036


In [ ]:
# Como salvar o modelo
torch.save(model.state_dict(), 'pep_lstm.pth')
files.download('pep_lstm.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Como carregar o modelo
model = PepModelLSTM().to(device)
model.load_state_dict(torch.load('pep_lstm_re_trained.pth'))
model.eval()

PepModelLSTM(
  (lstm): LSTM(20, 10, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)

In [ ]:
def sequence_to_one_hot(sequence, amino_acids):
    one_hot = torch.zeros(len(sequence), len(amino_acids))
    for i, aa in enumerate(sequence):
        one_hot[i, amino_acids.index(aa)] = 1
    return one_hot


def evaluate_sequences(model, sequences, amino_acids):
    probabilities = []
    with torch.no_grad():
        for sequence in sequences:
            input_sequence = sequence_to_one_hot(sequence, amino_acids)
            input_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0).to(device)
            output = model(input_tensor)
            probability = torch.sigmoid(output[:, 1]).item()
            probabilities.append(probability)
    return probabilities


def generate_random_sequences(num_sequences, sequence_length, amino_acids):
    sequences = []
    for _ in range(num_sequences):
        sequence = ''.join(random.choices(amino_acids, k=sequence_length))
        sequences.append(sequence)
    return sequences

num_sequences = 10  # Número de sequências que você quer gerar, altere conforme seu critério.
sequence_length = input("Qual o número de aminoácidos que sua sequência deve ter?"
                        "(Max=190)=> ")     # Nesse input, utilizamos o maior comprimento encontrado
sequence_length = int(sequence_length)
threshold = 0.99    # Probabilidade mínima utilizada, altere conforme seu critério.
high_prob_sequences = []

while len(high_prob_sequences) < num_sequences:
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    sequences = generate_random_sequences(num_sequences, sequence_length, amino_acids)
    probabilities = evaluate_sequences(model, sequences, amino_acids)

    for sequence, prob in zip(sequences, probabilities):
        if prob >= threshold:
            high_prob_sequences.append((sequence, prob))
            if len(high_prob_sequences) == num_sequences:
                break

high_prob_sequences_sorted = sorted(high_prob_sequences, key=lambda x: x[1], reverse=True)

print("Sequências com maior probabilidade AMP:")
for i, (sequence, prob) in enumerate(high_prob_sequences_sorted, start=1):
    print(f"{i}: Sequence: {sequence} \t Charge: {calc_net_charge(sequence)} \t <H>: {calc_hidrofob(sequence):.3f} \t Probability: {prob*100:.3f} %")

Qual o tamanho de sequência que deseja?(Max=190)=> 10
Sequências com maior probabilidade AMP:
1: Sequence: PVMFVQKAKK 	 Charge: 3 	 <H>: 0.330 	 Probability: 99.998 %
2: Sequence: PGQKHHVQGC 	 Charge: 1 	 <H>: 0.231 	 Probability: 99.998 %
3: Sequence: GGHKVSQSLG 	 Charge: 1 	 <H>: 0.176 	 Probability: 99.997 %
4: Sequence: FFCSKLNQPM 	 Charge: 1 	 <H>: 0.692 	 Probability: 99.995 %
5: Sequence: FNQYKKEAQK 	 Charge: 2 	 <H>: -0.159 	 Probability: 99.992 %
6: Sequence: GAFFGFMRDM 	 Charge: 0 	 <H>: 0.636 	 Probability: 99.990 %
7: Sequence: WWIGFDVWQM 	 Charge: -1 	 <H>: 1.180 	 Probability: 99.990 %
8: Sequence: WFGWKKDHAW 	 Charge: 1 	 <H>: 0.623 	 Probability: 99.987 %
9: Sequence: EWAGISQSLK 	 Charge: 0 	 <H>: 0.413 	 Probability: 99.933 %
10: Sequence: PIWLWVFKEW 	 Charge: 0 	 <H>: 1.235 	 Probability: 99.544 %


In [ ]:
# Nessa célula o usuário pode inserir uma sequência e ver sua probabilidade
user_sequence = input("Insira a sequência de aminoácidos: ")
user_probability = evaluate_sequences(model, [user_sequence], amino_acids)[0]
print(f"Probabilidade da sequência inserida ser AMP: {user_probability*100:.3f} %")

Insira a sequência de aminoácidos: FLSLIPKIASGAAKIAKHF
Probabilidade da sequência inserida ser AMP: 0.005 %


In [ ]:
torch.cuda.empty_cache()      # 'Limpar' a memória da GPU